<a href="https://colab.research.google.com/github/rana6-12/Final-Project-ITI/blob/main/Final_Project_ITI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ٌRana Yasser Anter**

**Upload Data Set**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd

# Replace with the actual path to your file in Google Drive
file_path = '/content/drive/MyDrive/IMDB Dataset.csv'

try:
  df = pd.read_csv(file_path)
  display(df.head())
except FileNotFoundError:
  print(f"Error: The file '{file_path}' was not found. Please check the path and try again.")
except Exception as e:
  print(f"An error occurred: {e}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.shape

In [ ]:
df.head(5)

**# Check columns, types, missing**

In [ ]:
print(df.columns)
print(df.dtypes)
print(df.isna().sum())

**# Class balance**

In [ ]:
print(df['sentiment'].value_counts())
sns.countplot(x='sentiment', data=df)
plt.title("Class distribution")
plt.show()

**# Length features**

In [ ]:
df['n_chars'] = df['review'].astype(str).str.len()
df['n_tokens'] = df['review'].astype(str).str.split().str.len()
print(df[['n_chars','n_tokens']].describe())

In [ ]:
plt.figure(figsize=(10,4))
sns.histplot(df['n_tokens'], bins=100, kde=True)
plt.title("Distribution of review token counts")
plt.show()

**Preprocessing :generic **

In [ ]:
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

**
# Download NLTK data once**

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

STOPWORDS = set(stopwords.words('english'))
LEMMATIZER = WordNetLemmatizer()

URL_RE = re.compile(r'https?://\S+|www\.\S+')
HTML_RE = re.compile(r'<.*?>')

In [ ]:
def generic_preprocess(text):
    # 1. lowercase
    t = str(text).lower()
    # 2. remove urls & html tags
    t = URL_RE.sub(' ', t)
    t = HTML_RE.sub(' ', t)
    # 3. remove punctuation
    t = t.translate(str.maketrans('', '', string.punctuation))
    # 4. keep letters only (optional) -> here we allow spaces and letters
    t = re.sub(r'[^a-z\s]', ' ', t)
    # 5. collapse spaces
    t = re.sub(r'\s+', ' ', t).strip()
    # 6. tokenize + remove stopwords + lemmatize
    tokens = [w for w in t.split() if w not in STOPWORDS]
    tokens = [LEMMATIZER.lemmatize(w) for w in tokens]
    return " ".join(tokens)

# Test
print(generic_preprocess(df['review'].iloc[0]))

**Preprocessing**

In [ ]:
import unicodedata




# normalize and remove non-letter characters (keeps spaces)

In [ ]:
def strip_non_letters(text):

    norm = unicodedata.normalize('NFKD', str(text))
    return re.sub(r'[^a-zA-Z\s]', ' ', norm)

 # remove emojis / special by keeping only letters and spaces

In [ ]:
def imdb_preprocess(text):
    t = str(text).lower()
    t = URL_RE.sub(' ', t)
    t = HTML_RE.sub(' ', t)
    t = strip_non_letters(t)
    t = re.sub(r'\s+', ' ', t).strip()
    tokens = [w for w in t.split() if w not in STOPWORDS]
    tokens = [LEMMATIZER.lemmatize(w) for w in tokens]
    return " ".join(tokens)

# Apply both pipelines and store columns

In [ ]:
df['clean_generic'] = df['review'].apply(generic_preprocess)
df['clean_imdb'] = df['review'].apply(imdb_preprocess)

# map sentiments to numbers

In [ ]:
df['label'] = df['sentiment'].map({'positive':1,'negative':0})

# show examples

In [ ]:
df[['review','clean_generic','clean_imdb','label']].head(6)

In [1]:
print("youssef tarek abdelmonem")


youssef tarek abdelmonem




> Add blockquote



youssef tarek abdelmonem
# compare length distributions before and after


In [ ]:
orig_tokens = df['review'].astype(str).str.split().str.len()
gen_tokens = df['clean_generic'].astype(str).str.split().str.len()
imdb_tokens = df['clean_imdb'].astype(str).str.split().str.len()

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(orig_tokens, bins=80, label='original', alpha=0.5)
sns.histplot(gen_tokens, bins=80, label='clean_generic', alpha=0.5)
sns.histplot(imdb_tokens, bins=80, label='clean_imdb', alpha=0.5)
plt.legend()
plt.title("Token length: before vs generic vs imdb-clean")
plt.show()


In [ ]:
from wordcloud import WordCloud

pos_text = " ".join(df[df['label']==1]['clean_imdb'].values)
neg_text = " ".join(df[df['label']==0]['clean_imdb'].values)


In [ ]:
wc_pos = WordCloud(width=800,height=400,background_color='white').generate(pos_text)
wc_neg = WordCloud(width=800,height=400,background_color='white').generate(neg_text)

plt.figure(figsize=(12,5))
plt.imshow(wc_pos); plt.axis('off'); plt.title("Positive Reviews Words")
plt.show()

plt.figure(figsize=(12,5))
plt.imshow(wc_neg); plt.axis('off'); plt.title("Negative Reviews Words")
plt.show()
